# Query Caching Effectiveness Report

This utility notebook analyzes the query cache hit rates. This is to ensure that caching is being used effectively and to reduce unnecessary compute costs.

Here's our 4 step process:
1. SQL query to retrieve data
2. Convert SQL table to a Pandas DataFrame
3. Data preparation and filtering (using user input from Streamlit widgets)
4. Data visualization and exploration

## 1. Retrieve Data

The following query filters for queries that actually scanned data, groups results by `WAREHOUSE_NAME`, and orders them by *percentage of data scanned from cache* (`percent_scanned_from_cache`). 

This helps to identify which warehouses are making the most effective use of caching.


In [ ]:
SELECT 
    warehouse_name,
    DATE_TRUNC('day', start_time) AS query_date,
    COUNT(DISTINCT query_parameterized_hash) AS query_parameterized_hash_count,
    COUNT(*) AS daily_executions,
    AVG(total_elapsed_time)/1000 AS avg_execution_time,
    SUM(total_elapsed_time)/1000 AS total_execution_time,
    SUM(CASE WHEN bytes_scanned > 0 THEN bytes_scanned ELSE 0 END) AS daily_bytes_scanned,
    SUM(bytes_scanned * percentage_scanned_from_cache) / NULLIF(SUM(CASE WHEN bytes_scanned > 0 THEN bytes_scanned ELSE 0 END), 0) AS daily_cache_hit_ratio,
    MAX_BY(query_text, start_time) AS latest_query_text,
    MAX_BY(user_name, start_time) AS latest_user_name
FROM snowflake.account_usage.query_history qh
WHERE start_time >= dateadd(day, -30, current_timestamp())
GROUP BY 1, 2
HAVING daily_bytes_scanned > 0
ORDER BY 
    query_date DESC,
    daily_cache_hit_ratio DESC,
    daily_bytes_scanned DESC

## 2. Convert Table to a DataFrame

Next, we'll convert the tables to a Pandas DataFrame.


In [ ]:
sql_query_caching.to_pandas()

## Data Preparation

Here, we'll do some data preparation prior to visualization.

In [ ]:
df = py_query_caching.copy()

# Convert QUERY_DATE to datetime
df['QUERY_DATE'] = pd.to_datetime(df['QUERY_DATE'])

# Create WEEK_NUMBER column
df['WEEK_NUMBER'] = df['QUERY_DATE'].dt.isocalendar().week

# Create MONTH_YEAR column
df['MONTH_YEAR'] = df['QUERY_DATE'].dt.strftime('%b %Y')

# Group by
grouped_df = df.groupby('WAREHOUSE_NAME').agg({
    'QUERY_PARAMETERIZED_HASH_COUNT': 'count',
    'DAILY_EXECUTIONS': 'sum',
    'AVG_EXECUTION_TIME': 'mean',
    'TOTAL_EXECUTION_TIME': 'sum',
    'DAILY_BYTES_SCANNED': 'sum',
    'DAILY_CACHE_HIT_RATIO': 'mean'
}).reset_index()

grouped_df

## 3. Visualize Bar Chart

Here, we'll visualize the data via a bar chart for the columns:
- Query count
- Bytes scanned
- Percent of bytes scanned


In [ ]:
import altair as alt
import pandas as pd

# Create bar chart
chart = alt.Chart(grouped_df).mark_bar().encode(
    y=alt.Y('WAREHOUSE_NAME:N', 
            title='',
            axis=alt.Axis(
                labels=True,
                labelLimit=250,
                tickMinStep=1,
                labelOverlap=False,
                labelPadding=10
            ),
            sort='-x'),
    x=alt.X('DAILY_CACHE_HIT_RATIO:Q', 
            title='Cache Hit Ratio'),
    color=alt.Color('WAREHOUSE_NAME:N', legend=None),
    tooltip=[
        alt.Tooltip('WAREHOUSE_NAME', title='Warehouse'),
        alt.Tooltip('DAILY_CACHE_HIT_RATIO', title='Cache Hit Ratio'),
        alt.Tooltip('DAILY_EXECUTIONS', title='Daily Executions'),
        alt.Tooltip('AVG_EXECUTION_TIME', title='Avg Execution Time (ms)')
    ]
).properties(
    width=400,
    height=600,
    title='Cache Hit Ratio by Warehouse'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16,
    anchor='middle'
)

# Display the chart
st.altair_chart(chart, use_container_width=True)

## 4. Visualize as Heatmap

Here, we'll visualize the data via a heatmap for the columns:
- Query count
- Bytes scanned
- Percent of bytes scanned


In [ ]:
import pandas as pd
import altair as alt

# Convert QUERY_DATE to datetime if it isn't already
df['QUERY_DATE'] = pd.to_datetime(df['QUERY_DATE'])

# Format date as string for display
df['DATE'] = df['QUERY_DATE'].dt.strftime('%Y-%m-%d')

# Aggregate data by date and warehouse
agg_df = df.groupby(['DATE', 'WAREHOUSE_NAME'])['DAILY_CACHE_HIT_RATIO'].sum().reset_index()

# Create the heatmap
heatmap = alt.Chart(agg_df).mark_rect(stroke='black', strokeWidth=1).encode(
   x=alt.X('DATE:O',
           title='Date',
           axis=alt.Axis(
               labelAngle=90,
               labelOverlap=False,
               tickCount=10
           )),
   y=alt.Y('WAREHOUSE_NAME:N',
           title='',
           axis=alt.Axis(
               labels=True,
               labelLimit=250,
               tickMinStep=1,
               labelOverlap=False,
               labelPadding=10
           )),
   color=alt.Color('DAILY_CACHE_HIT_RATIO:Q',
                   title='Cache Hit Ratio',
                   scale=alt.Scale(scheme='blues')),
   tooltip=['DATE', 'WAREHOUSE_NAME', 
           alt.Tooltip('DAILY_CACHE_HIT_RATIO:Q', format='.2%')]
).properties(
   title=f'Daily Warehouse Cache Hit Ratio Heatmap',
   width=500,
   height=600
)

# Add configuration to make the chart more interactive
heatmap = heatmap.configure_axis(
   grid=False
).configure_view(
   strokeWidth=0
)

# Display or save the chart
st.altair_chart(heatmap, use_container_width=True)

💡 Pro tip:

When you see a low cache scan percentage for queries that repeatedly access the same data, you can significantly improve its performance by optimizing the cache usage. This is especially true for reports or dashboards that run similar queries throughout the day.

## Want to learn more?

- Snowflake Docs on [Account Usage](https://docs.snowflake.com/en/sql-reference/account-usage) and [QUERY_HISTORY view](https://docs.snowflake.com/en/sql-reference/account-usage/query_history)
- More about [Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-use-with-snowflake)
- For more inspiration on how to use Streamlit widgets in Notebooks, check out [Streamlit Docs](https://docs.streamlit.io/) and this list of what is currently supported inside [Snowflake Notebooks](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-use-with-snowflake#label-notebooks-streamlit-support)
- Check out the [Altair User Guide](https://altair-viz.github.io/user_guide/data.html) for further information on customizing Altair charts
